<a href="https://colab.research.google.com/github/Navanithapatel/GEN-AI-workshop/blob/main/RAG_AGENTAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q numpy==1.26.4 sentence-transformers transformers faiss-cpu pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have

In [1]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from pypdf import PdfReader
import faiss
import numpy as np


In [2]:
from google.colab import files
uploaded = files.upload()


Saving Unit - 1_Introduction to Internet of  Things.pdf to Unit - 1_Introduction to Internet of  Things.pdf


In [3]:
file_name = list(uploaded.keys())[0]

reader = PdfReader(file_name)
text = ""

for page in reader.pages:
    page_text = page.extract_text()
    if page_text:
        text += page_text

print("PDF loaded successfully")


PDF loaded successfully


In [4]:
def split_text(text, chunk_size=400):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunks.append(" ".join(words[i:i+chunk_size]))
    return chunks

chunks = split_text(text)
print("Total chunks:", len(chunks))


Total chunks: 2


In [5]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

print("Vector database created successfully")


Vector database created successfully


In [7]:
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=200
)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [8]:
def rag_answer(question, k=3):
    q_embedding = embedder.encode([question])
    distances, indices = index.search(np.array(q_embedding), k)

    context = " ".join([chunks[i] for i in indices[0]])

    prompt = f"""
    Answer the question using the context below.
    Context: {context}
    Question: {question}
    """

    return generator(prompt)[0]["generated_text"]


In [12]:
class NotesAIAgent:
    def __init__(self):
        self.goal_achieved = False

    def run(self, question):
        attempts = 0
        while not self.goal_achieved and attempts < 3:
            print(f"\n Attempt {attempts + 1}")
            answer = rag_answer(question)
            print("Answer:", answer)

            if len(answer.strip()) > 30:
                self.goal_achieved = True

            attempts += 1

        print("\n Goal Achieved")
        return answer


In [15]:
agent = NotesAIAgent()
agent.run("Explain physical diagram")




 Attempt 1
Answer: Physical Design of IoT  Physical design refers to the hardware components of an IoT system that can be physically seen and touched.  It includes devices, sensors, actuators, communication modules, and power supply units.  Physical design explains how devices are built, connected, and used to collect and control data.

 Goal Achieved


'Physical Design of IoT  Physical design refers to the hardware components of an IoT system that can be physically seen and touched.  It includes devices, sensors, actuators, communication modules, and power supply units.  Physical design explains how devices are built, connected, and used to collect and control data.'

In [16]:
#for multiple
class NotesAIAgent:
    def __init__(self):
        pass

    def run(self, question):
        goal_achieved = False
        attempts = 0
        answer = ""

        while not goal_achieved and attempts < 3:
            print(f"\n Attempt {attempts + 1}")
            answer = rag_answer(question)
            print("Answer:", answer)

            if len(answer.strip()) > 30:
                goal_achieved = True

            attempts += 1

        print("\n Goal Achieved")
        return answer


In [17]:
agent = NotesAIAgent()

agent.run("Explain physical design of IoT")
agent.run("What is IoT?")
agent.run("Explain characteristics of IoT")
agent.run("What are sensors and actuators?")




🤖 Attempt 1
Answer: Physical design refers to the hardware components of an IoT system that can be physically seen and touched.

✅ Goal Achieved

🤖 Attempt 1
Answer: The Internet of Things (IoT) is a dynamic global network in which physical and virtual objects are connected through the internet using standard communication protocols.

✅ Goal Achieved

🤖 Attempt 1
Answer: Connectivity:  IoT devices are connected to the internet or a network so that data can be shared between devices and systems. Sensing:  Sensors collect real-world information such as temperature, humidity, or motion and convert it into digital data. Automation:  IoT systems can perform tasks automatically based on collected data, reducing the need for manual operation.Intelligence: Data is analyzed using software to make decisions, such as turning ON a fan when temperature increases. Security: Since data is transmitted over the internet, protecting data from unauthorized access is very important.

✅ Goal Achieved

🤖 A

'input devices that collect information from the environment'